In [1]:
%load_ext autotime
import numpy as np
from keras.layers import Input, Dense, Conv1D,MaxPooling1D,UpSampling1D ,K,AveragePooling1D 
from keras.models import Model,Sequential
from keras.datasets import mnist
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
a=np.genfromtxt('data_for_clustering.csv', delimiter=',')
a=a[:-1]
print(a[1])
x=a[:250*910]
x=x[20*250:]
x=x.reshape(-1,250)
me=np.mean(x,axis=1)
x=(x.T-me).T
x=x.reshape(890,250,1)
x.shape

39963.59


(890, 250, 1)

time: 2.49 s


In [3]:
model=Sequential()
pool_size=10
#model.add(Dense(250, activation='linear'))
model.add(Conv1D(4,10, input_shape=x.shape[1:],activation='linear',padding='same'))
model.add(AveragePooling1D(pool_size=(5),padding='same'))

model.add(Conv1D(2,10,padding='same',activation='linear'))
model.add(AveragePooling1D(pool_size=(5),padding='same'))

#model.add(Conv1D(1,5,padding='same',activation='linear'))
#model.add(AveragePooling1D(pool_size=(5),padding='same'))

#model.add(Conv1D(1,5,padding='same',activation='linear'))
#model.add(UpSampling1D(5))

model.add(Conv1D(2,10,padding='same',activation='linear'))
model.add(UpSampling1D(5))

model.add(Conv1D(4,10,padding='same',activation='linear'))
model.add(UpSampling1D(5))
model.add(Dense(250, activation='linear'))
model.add(Conv1D(1,10, padding='same',activation='linear'))


model.compile(loss='mse',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
time: 213 ms


In [4]:
model.fit(x, x, batch_size=10, epochs=50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
890/890 [==============================] - 5s 5ms/step - loss: 44275.7740 - acc: 0.0000e+00
Epoch 2/50
890/890 [==============================] - 1s 834us/step - loss: 40306.3872 - acc: 0.0000e+00
Epoch 3/50
890/890 [==============================] - 1s 835us/step - loss: 39060.4055 - acc: 0.0000e+00
Epoch 4/50
890/890 [==============================] - 1s 798us/step - loss: 38266.1245 - acc: 0.0000e+00
Epoch 5/50
890/890 [==============================] - 1s 831us/step - loss: 37375.3788 - acc: 0.0000e+00
Epoch 6/50
890/890 [==============================] - 1s 891us/step - loss: 35628.5366 - acc: 0.0000e+00
Epoch 7/50
890/890 [==============================] - 1s 831us/step - loss: 32490.9278 - acc: 0.0000e+00
Epoch 8/50
890/890 [==============================] - 1s 830us/step - loss: 27965.5679 - acc: 0.0000e+00
Epoch 9/50
890/890 [==============================] - 1s 879us/step - loss: 24515.3178 - acc: 0.0000e+00
Epoch 10/

time: 41.2 s


In [5]:
model.get_config()['layers']

[{'class_name': 'Conv1D',
  'config': {'name': 'conv1d_1',
   'trainable': True,
   'batch_input_shape': (None, 250, 1),
   'dtype': 'float32',
   'filters': 4,
   'kernel_size': (10,),
   'strides': (1,),
   'padding': 'same',
   'data_format': 'channels_last',
   'dilation_rate': (1,),
   'activation': 'linear',
   'use_bias': True,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'scale': 1.0,
     'mode': 'fan_avg',
     'distribution': 'uniform',
     'seed': None}},
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'kernel_regularizer': None,
   'bias_regularizer': None,
   'activity_regularizer': None,
   'kernel_constraint': None,
   'bias_constraint': None}},
 {'class_name': 'AveragePooling1D',
  'config': {'name': 'average_pooling1d_1',
   'trainable': True,
   'strides': (5,),
   'pool_size': (5,),
   'padding': 'same',
   'data_format': 'channels_last'}},
 {'class_name': 'Conv1D',
  'config': {'name': 'conv1d_2',
   'trainable': True,

time: 5.96 ms


In [6]:
compressed_layer =3
get_3rd_layer_output = K.function([model.layers[0].input], [model.layers[compressed_layer].output])
compressed = get_3rd_layer_output([x])[0]

time: 82.8 ms


In [7]:
compressed[0].shape

(10, 2)

time: 2 ms


In [8]:
decoded_output=model.predict(x)

time: 207 ms


In [9]:
decoded_output[0]

array([[-7.29799194e+01],
       [-1.22004768e+02],
       [-1.71493408e+02],
       [-2.17806992e+02],
       [-3.20966705e+02],
       [-2.88303650e+02],
       [-2.66253021e+02],
       [-2.38690796e+02],
       [-2.11015564e+02],
       [-1.72397964e+02],
       [-1.33988678e+02],
       [-9.74002151e+01],
       [-6.50422134e+01],
       [-3.57819481e+01],
       [ 7.86783838e+00],
       [ 2.92193241e+01],
       [ 4.35110321e+01],
       [ 5.54203911e+01],
       [ 7.08895493e+01],
       [ 1.10565598e+02],
       [ 1.63939926e+02],
       [ 1.95748016e+02],
       [ 2.18790955e+02],
       [ 2.46017395e+02],
       [ 3.15432220e+02],
       [ 2.90783783e+02],
       [ 2.82143433e+02],
       [ 2.69876617e+02],
       [ 2.54723389e+02],
       [ 2.35608856e+02],
       [ 2.19081696e+02],
       [ 1.89044952e+02],
       [ 1.62729797e+02],
       [ 1.44339737e+02],
       [ 1.33913208e+02],
       [ 1.30857361e+02],
       [ 1.35994690e+02],
       [ 1.44403030e+02],
       [ 1.6

time: 4.97 ms


In [11]:
import matplotlib.pyplot as plt
%matplotlib widget
q=10
plt.plot(range(250),x[q],label=1)
plt.plot(range(250),decoded_output[q],label=2)

FigureCanvasNbAgg()

time: 31.9 ms


In [20]:
compressed=compressed.reshape(890,20)
np.savetxt('compressed_data1.csv',compressed,delimiter=',')

time: 45.9 ms


In [19]:
print(compressed)
print(compressed.reshape(890,20))

[[[145.64311    89.69908  ]
  [ 59.87969   -75.12716  ]
  [-47.12908   -18.28421  ]
  ...
  [-16.267395  -19.610474 ]
  [-73.05257   -16.538263 ]
  [ -2.4309487 -51.38665  ]]

 [[-24.813282  -60.113045 ]
  [-30.544958   30.06396  ]
  [105.22003   -11.0095825]
  ...
  [-19.988525   89.35518  ]
  [ -2.4909637 -41.62992  ]
  [ -9.945433  -60.893238 ]]

 [[ 24.401154  -20.687405 ]
  [-23.201563   12.746546 ]
  [-73.02566   -24.93848  ]
  ...
  [ -9.8739195 -61.45342  ]
  [-35.201363  -34.90452  ]
  [ 11.596093   31.951382 ]]

 ...

 [[-23.3797    -51.320454 ]
  [ 23.547718    5.016565 ]
  [119.63274   -27.92373  ]
  ...
  [-61.158913  -39.94712  ]
  [-41.478436   82.67122  ]
  [ -1.2712152 -29.121058 ]]

 [[-19.346819   30.418798 ]
  [-20.674006  -18.746948 ]
  [107.348465   20.43299  ]
  ...
  [-35.73685    24.40514  ]
  [ 32.35891    61.52555  ]
  [-17.708881  -52.101227 ]]

 [[-11.90168    -2.028717 ]
  [ 18.493996   55.24243  ]
  [  7.5473967 -36.094616 ]
  ...
  [-63.115124    2.83878